## COD API, One Match --> explore API, format, clean & reshape

Activision Call of Duty API use case for **Match** endpoint, using a slightly amended version of callofduty.py client and custom cleaning functions (from wzkd app)

In [1]:
import asyncio
import os
import sys
import dotenv
from pprint import pprint
import datetime
from datetime import datetime, timezone, timedelta
import pandas as pd
import numpy as np
import pickle

import callofduty
from callofduty import Mode, Platform, Title, TimeFrame, GameType, Language

#### Login using SSO

In [3]:
# We're storing our SSO token in an .env file stored locally to separate our conf from code (w. python-dotenv). An.env-template file (with help to retrieve token) is provided for you to edit and populate the variable(s)
# callofduty.py client .Login() goes through all the authentification steps and initiate a session to access protected routes
# The client is asynchronous thus the 'await style'
from dotenv import load_dotenv
load_dotenv()
client = await callofduty.Login(sso=os.environ["SSO"])

#### Slightly modify client method to call a given match

In [4]:
# 1. import client classes
import urllib.parse
from typing import List, Optional, Union

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request

# 2. define a modified method
async def GetMatchStats(
    self, platform, title: Title, mode: Mode, matchId: int, language: Language = Language.English, **kwargs
):
    """ 
    Compared to client : modified so that we do not use Platform.abc as parameter
    but instead our app-defined workflow (drop down menu) to select our platform of choice"
    """
    return (
        await self.http.GetFullMatch(
            title.value, platform, mode.value, matchId, language.value
        )
    )["data"]["allPlayers"]
    # api result, at very least for Warzone {'data':{'all_players:' is the only key},'status': call status}

# 3. add modified method in callofduty.py client at runtime
Client.GetMatchStats = GetMatchStats

#### Get Match data

In [7]:
match = await client.GetMatchStats('battle', Title.ModernWarfare, Mode.Warzone, matchId=10717821121770145230)

# example of IDs :
# 10717821121770145230 : BR duos, Season 5 april 2022
# 1245830289127567228 : BR trios, with me 6 kills
# 5010761031247628577 : BR trios 
# 11672696746036290501 a "custom" mode game (Rumble Clash)
# 6825832239054239925 : Iron Trials Trios

##### Option: save previous result so we're not getting annoyed by API rate limits or inconsistencies -,-

In [8]:
# save previous result so we're not getting annoyed by rate limits or API inconsistencies
with open("match.pkl", 'wb') as f:
    pickle.dump(match, f)

## Match result : structure

In [9]:
# load previously saved data
with open('match.pkl', 'rb') as f:
    match = pickle.load(f)

### Overview : dict --> df

In [10]:
df_match = pd.DataFrame(match)
display(df_match.head(2))
keys = list(df_match.keys())
keys.sort()
pprint(keys)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1650402208,1650403856,mp_wz_island,br_brduos,10717821121770145230,1648000,None,1,wz,156,"{'kills': 0.0, 'medalXp': 0.0, 'matchXp': 1177...","{'team': 'team_sixty_three', 'rank': 54.0, 'aw...",75,None,False,False
1,1650402208,1650403856,mp_wz_island,br_brduos,10717821121770145230,1648000,None,1,wz,156,"{'kills': 0.0, 'medalXp': 0.0, 'matchXp': 1358...","{'team': 'team_thirty_five', 'rank': 54.0, 'aw...",75,None,False,False


['draw',
 'duration',
 'gameType',
 'map',
 'matchID',
 'mode',
 'player',
 'playerCount',
 'playerStats',
 'playlistName',
 'privateMatch',
 'rankedTeams',
 'teamCount',
 'utcEndSeconds',
 'utcStartSeconds',
 'version']


### API structure for one player (a 'row')

##### Reminder : match endpoint returns n players stats (typically 150+ in Battle Royale) as rows, for a single --queried match

In [11]:
pprint(match[0:1], depth=3)

[{'draw': False,
  'duration': 1648000,
  'gameType': 'wz',
  'map': 'mp_wz_island',
  'matchID': '10717821121770145230',
  'mode': 'br_brduos',
  'player': {'awards': {},
             'brMissionStats': {...},
             'clantag': 'WS021',
             'loadout': [...],
             'loadouts': [...],
             'rank': 54.0,
             'team': 'team_sixty_three',
             'uno': '14304991762383140659',
             'username': 'Mike'},
  'playerCount': 156,
  'playerStats': {'assists': 1.0,
                  'bonusXp': 0.0,
                  'challengeXp': 0.0,
                  'damageDone': 292.0,
                  'damageTaken': 816.0,
                  'deaths': 3.0,
                  'distanceTraveled': 421910.38,
                  'executions': 0.0,
                  'gulagDeaths': 2.0,
                  'gulagKills': 0.0,
                  'headshots': 0.0,
                  'kdRatio': 0.0,
                  'kills': 0.0,
                  'longestStreak': 0.0,
     

### Focus : what's in 'playerStats' ?

In [12]:
player_stats = df_match['playerStats'].apply(pd.Series)
display(player_stats.head(5))
pprint(player_stats.keys())

,kills,medalXp,matchXp,scoreXp,wallBangs,score,totalXp,headshots,assists,challengeXp,...,timePlayed,executions,gulagKills,nearmisses,percentTimeMoving,miscXp,longestStreak,teamPlacement,damageDone,damageTaken
0,0.0,0.0,11778.0,2325.0,0.0,2025.0,14136.0,0.0,1.0,0.0,...,1454.0,0.0,0.0,0.0,62.316715,0.0,0.0,5.0,292.0,816.0
1,0.0,0.0,1358.0,0.0,0.0,0.0,1358.0,0.0,0.0,0.0,...,287.0,0.0,0.0,0.0,49.065420,0.0,0.0,67.0,208.0,450.0
2,1.0,10.0,5307.0,750.0,0.0,750.0,6067.0,0.0,0.0,0.0,...,575.0,0.0,1.0,0.0,52.845528,0.0,1.0,38.0,444.0,1030.0
3,3.0,40.0,10702.0,1750.0,0.0,1350.0,12753.0,0.0,0.0,0.0,...,1267.0,0.0,0.0,0.0,87.935430,0.0,3.0,9.0,1014.0,313.0
4,0.0,0.0,6625.0,1370.0,0.0,725.0,7995.0,0.0,1.0,0.0,...,738.0,0.0,0.0,0.0,63.636364,0.0,0.0,31.0,109.0,600.0


Index(['kills', 'medalXp', 'matchXp', 'scoreXp', 'wallBangs', 'score',
       'totalXp', 'headshots', 'assists', 'challengeXp', 'rank',
       'scorePerMinute', 'distanceTraveled', 'teamSurvivalTime', 'deaths',
       'kdRatio', 'bonusXp', 'gulagDeaths', 'timePlayed', 'executions',
       'gulagKills', 'nearmisses', 'percentTimeMoving', 'miscXp',
       'longestStreak', 'teamPlacement', 'damageDone', 'damageTaken'],
      dtype='object')


In [12]:
# some cols may not appear re. the mode you're playing in
cols_set1 = [
    'kills',
    'medalXp',
    'score',
    'matchXp',
    'scoreXp',
    'scorePerMinute',
    'challengeXp',
    'totalXp'
]
cols_set2 = [
    'kills',
     'deaths',
     'assists',
     'kdRatio',
     'headshots',
     'percentTimeMoving',
     'nearmisses',
     'executions',
     'longestStreak',
     'damageDone',
     'damageTaken'
]
cols_set3 = [
    'kills',
    'timePlayed',
    'distanceTraveled',
    'teamSurvivalTime',
    'teamPlacement'
]
cols_set4 = [
    'kills',
    'objectiveTeamWiped',
    'objectiveLastStandKill',
    'objectiveBrDownEnemyCircle1',
    'objectiveBrDownEnemyCircle2',
    'objectiveBrDownEnemyCircle3',
    'objectiveBrDownEnemyCircle4',
    'objectiveBrDownEnemyCircle5',
    'objectiveBrDownEnemyCircle6',
]
cols_set5 = [
    'kills',
    'objectiveReviver',
    'objectiveBrMissionPickupTablet',
    'objectiveBrCacheOpen',
    'objectiveBrKioskBuy'
]
cols_set5  = [
    'kills',
    'objectiveMunitionsBoxTeammateUsed',
    'objectiveMedalScoreSsKillTomaStrike',
    'objectiveDestroyedVehicleHeavy',
    'objectiveDestroyedVehicleMedium'
]

In [13]:
for set_cols in [cols_set1, cols_set2, cols_set3, cols_set4, cols_set5]:
    # some colums (keys) not existing depending on gamemode /  or all values missing
    set_cols = [col for col in set_cols if col in player_stats.columns.tolist()] 
    display(player_stats[set_cols].sort_values('kills', ascending=False).head(10))

,kills,medalXp,score,matchXp,scoreXp,scorePerMinute,challengeXp,totalXp
9,13.0,850.0,13250.0,12796.0,14270.0,481.234867,0.0,30841.0
116,13.0,860.0,14975.0,12471.0,15820.0,571.928708,0.0,33682.0
41,11.0,345.0,7550.0,12305.0,7850.0,269.482451,0.0,22787.0
84,9.0,310.0,4600.0,9134.0,5600.0,231.155779,0.0,15890.0
58,8.0,335.0,9500.0,9134.0,10500.0,476.987448,0.0,20248.0
79,8.0,290.0,3200.0,5298.0,3575.0,251.638270,0.0,9217.0
130,7.0,50.0,5250.0,12941.0,7090.0,187.835420,0.0,21734.0
30,7.0,290.0,8925.0,11969.0,9325.0,344.372990,0.0,23375.0
78,7.0,445.0,9275.0,12941.0,11515.0,331.842576,0.0,25816.0
5,7.0,325.0,3075.0,4548.0,3175.0,273.333333,0.0,8098.0


,kills,deaths,assists,kdRatio,headshots,percentTimeMoving,nearmisses,executions,longestStreak,damageDone,damageTaken
9,13.0,2.0,1.0,6.500000,4.0,95.191040,0.0,0.0,11.0,5314.0,1005.0
116,13.0,3.0,8.0,4.333333,3.0,93.561360,0.0,0.0,10.0,5786.0,1664.0
41,11.0,2.0,3.0,5.500000,1.0,71.805010,0.0,0.0,7.0,4804.0,550.0
84,9.0,4.0,1.0,2.250000,2.0,93.525185,0.0,0.0,6.0,2845.0,1597.0
58,8.0,2.0,3.0,4.000000,0.0,87.758780,0.0,0.0,8.0,3209.0,1040.0
79,8.0,2.0,0.0,4.000000,2.0,91.569760,0.0,0.0,7.0,3849.0,584.0
130,7.0,2.0,1.0,3.500000,0.0,83.662710,0.0,0.0,4.0,2996.0,901.0
30,7.0,2.0,3.0,3.500000,0.0,72.059820,0.0,0.0,7.0,3714.0,990.0
78,7.0,2.0,0.0,3.500000,2.0,89.262184,0.0,0.0,5.0,2417.0,1338.0
5,7.0,2.0,1.0,3.500000,2.0,82.500000,0.0,0.0,6.0,2780.0,780.0


,kills,timePlayed,distanceTraveled,teamSurvivalTime,teamPlacement
9,13.0,1652.0,363130.75,1534704.0,2.0
116,13.0,1571.0,268589.28,1488432.0,3.0
41,11.0,1681.0,481859.75,1483200.0,4.0
84,9.0,1194.0,373465.94,1116144.0,16.0
58,8.0,1195.0,523171.66,1116144.0,16.0
79,8.0,763.0,285987.38,690288.0,31.0
130,7.0,1677.0,366049.30,1534704.0,1.0
30,7.0,1555.0,345387.38,1471248.0,6.0
78,7.0,1677.0,390530.60,1534704.0,1.0
5,7.0,675.0,351686.47,609456.0,34.0


,kills,objectiveTeamWiped,objectiveLastStandKill,objectiveBrDownEnemyCircle1
9,13.0,NaN,NaN,NaN
116,13.0,NaN,NaN,NaN
41,11.0,NaN,NaN,NaN
84,9.0,NaN,NaN,NaN
58,8.0,NaN,NaN,NaN
79,8.0,NaN,NaN,NaN
130,7.0,NaN,NaN,NaN
30,7.0,NaN,NaN,NaN
78,7.0,NaN,NaN,NaN
5,7.0,2.0,4.0,6.0


,kills
9,13.0
116,13.0
41,11.0
84,9.0
58,8.0
79,8.0
130,7.0
30,7.0
78,7.0
5,7.0


Quick notes :</br>
- objectiveTeamWiped = teams wiped
- objectiveLastStandKill = kills when alone in team ?? ^_o

### Focus : 'player', a nested entry

In [13]:
player = df_match['player'].apply(pd.Series)
display(player.head(5))
pprint(player.keys())

,team,rank,awards,username,uno,clantag,loadouts,brMissionStats,loadout
0,team_sixty_three,54.0,{},Mike,14304991762383140659,WS021,"[{'primaryWeapon': {'name': 's4_ar_hyankee44',...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 's4_ar_hyankee44',..."
1,team_thirty_five,54.0,{},Tedgar,12168037821978496320,THIS,"[{'primaryWeapon': {'name': 'iw8_sn_kilo98', '...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sn_kilo98', '..."
2,team_sixteen,54.0,"{'low_health_kill': 76944.0, 'mode_x_eliminate...",derGENTLEMAN,16316355057210015885,NaN,"[{'primaryWeapon': {'name': 's4_ar_hyankee44',...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 's4_ar_hyankee44',..."
3,team_sixty,35.0,"{'low_health_kill': 1120368.0, 'mode_x_elimina...",Buddhaseijin,16661802044109520739,NaN,"[{'primaryWeapon': {'name': 'iw8_lm_mkilo3', '...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_lm_mkilo3', '..."
4,team_thirty_three,54.0,{},TMONEY,3829679737724644663,THE,[{'primaryWeapon': {'name': 'iw8_ar_t9standard...,"{'missionsComplete': 1, 'totalMissionXpEarned'...",[{'primaryWeapon': {'name': 'iw8_ar_t9standard...


Index(['team', 'rank', 'awards', 'username', 'uno', 'clantag', 'loadouts',
       'brMissionStats', 'loadout'],
      dtype='object')


#### Inside 'player' entry, 'loadout' is a (list of) list of dict

In [17]:
# Each entry of 'loadout' (or loadouts, they are the same) is a list of dict. Either one dict (if 1 loadout) or more (if you succeed in buying several loadouts)
player_index = 2
pprint(player['loadout'][player_index][0], depth=2)

{'extraPerks': [{...}, {...}, {...}],
 'killstreaks': [{...}, {...}, {...}],
 'lethal': {'image': None,
            'imageLarge': None,
            'label': None,
            'name': 'equip_semtex',
            'progressionImage': None},
 'perks': [{...}, {...}, {...}],
 'primaryWeapon': {'attachments': [...],
                   'imageIcon': None,
                   'imageLoot': None,
                   'label': None,
                   'name': 's4_ar_hyankee44',
                   'variant': '0'},
 'secondaryWeapon': {'attachments': [...],
                     'imageIcon': None,
                     'imageLoot': None,
                     'label': None,
                     'name': 's4_sm_ppapa41',
                     'variant': '0'},
 'tactical': {'image': None,
              'imageLarge': None,
              'label': None,
              'name': 'equip_hb_sensor',
              'progressionImage': None}}


#### Inside 'player' entry, 'brMissionStats' is a (list of) dict

In [14]:
player['brMissionStats'].apply(pd.Series).head(5)

,missionsComplete,totalMissionXpEarned,totalMissionWeaponXpEarned,missionStatsByType
0,0,0.0,0.0,{}
1,0,0.0,0.0,{}
2,0,0.0,0.0,{}
3,0,0.0,0.0,{}
4,1,300.0,300.0,"{'assassination': {'weaponXp': 300.0, 'xp': 30..."


In [15]:
player['brMissionStats'].apply(pd.Series)[['missionStatsByType']]['missionStatsByType'].apply(pd.Series).head(10)

,assassination,scavenger,domination,supply,timedrun,masterassassination
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'weaponXp': 300.0, 'xp': 300.0, 'count': 1.0}",NaN,NaN,NaN,NaN,NaN
5,"{'weaponXp': 1000.0, 'xp': 1000.0, 'count': 1.0}",NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,"{'weaponXp': 575.0, 'xp': 575.0, 'count': 1.0}",NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
with open('match.pkl', 'rb') as f:
    match = pickle.load(f)
df_match = pd.DataFrame(match)
df_loadouts = extract_loadouts(df_match)
df_loadouts.head(2)

,loadout_1,loadout_2,loadout_3
0,"{'primaryWeapon': {'name': 's4_pi_mike1911', '...","{'primaryWeapon': {'name': 's4_ar_fecho', 'lab...",NaN
1,"{'primaryWeapon': {'name': 'iw8_ar_mike4', 'la...",NaN,NaN


## Format & clean API **match** results using customized tools (wzkd app)

#### Inside 'player' entry, 'loadout' is a (list of) list of dict

In [2]:
import json
import toml
# functions defined in wzkd app directory '/wzkd/wzkd'
sys.path.insert(0, os.path.abspath('../wzkd'))
from utils import load_labels, load_conf
from api_format import res_to_df, format_df

In [3]:
# conf and labels files stored here as well.
# labels is needed for parsing games modes/weapons, conf stores values such as n of loadouts to extract or columns names
file_labels = "wz_labels.json"
filepath_labels = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+ "/wzkd/" + file_labels
LABELS = load_labels(filepath_labels)
pprint(LABELS, depth=2)

file_conf = "conf.toml"
filepath_conf = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+ "/wzkd/" + file_conf
CONF = load_conf(filepath_conf)
pprint(CONF, depth=2)

{'modes': {'battle_royale': {...}, 'multiplayer': {...}},
 'weapons': {'cat_names': {...},
             'categories': [...],
             'names': {...},
             'prefixes': [...]}}
{'API_OUTPUT_FORMAT': {'float_cols': [...],
                       'int_cols': [...],
                       'mission_types': [...],
                       'n_loadouts': 3,
                       'ts_cols': [...]},
 'APP_BEHAVIOR': {'br_only': True, 'mode': 'offline'},
 'APP_DISPLAY': {'keep_cols': {...}, 'labels': {...}}}


In [10]:
with open('match.pkl', 'rb') as f:
    res = pickle.load(f)
tmp = pd.DataFrame(res)
display(tmp.head(2))
tmp.info()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1650402208,1650403856,mp_wz_island,br_brduos,10717821121770145230,1648000,None,1,wz,156,"{'kills': 0.0, 'medalXp': 0.0, 'matchXp': 1177...","{'team': 'team_sixty_three', 'rank': 54.0, 'aw...",75,None,False,False
1,1650402208,1650403856,mp_wz_island,br_brduos,10717821121770145230,1648000,None,1,wz,156,"{'kills': 0.0, 'medalXp': 0.0, 'matchXp': 1358...","{'team': 'team_thirty_five', 'rank': 54.0, 'aw...",75,None,False,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   utcStartSeconds  156 non-null    int64 
 1   utcEndSeconds    156 non-null    int64 
 2   map              156 non-null    object
 3   mode             156 non-null    object
 4   matchID          156 non-null    object
 5   duration         156 non-null    int64 
 6   playlistName     0 non-null      object
 7   version          156 non-null    int64 
 8   gameType         156 non-null    object
 9   playerCount      156 non-null    int64 
 10  playerStats      156 non-null    object
 11  player           156 non-null    object
 12  teamCount        156 non-null    int64 
 13  rankedTeams      0 non-null      object
 14  draw             156 non-null    bool  
 15  privateMatch     156 non-null    bool  
dtypes: bool(2), int64(6), object(8)
memory usage: 17.5+ KB


In [11]:
# flatten-expand into a DataFrame the result from COD API, for matches history
df_match = res_to_df(res, CONF)
display(df_match.head(3))
df_match.info()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,version,gameType,playerCount,teamCount,...,loadout_3,missionsComplete,totalMissionXpEarned,totalMissionWeaponXpEarned,assassination,scavenger,domination,supply,timedrun,masterassassination
0,1650402208,1650403856,mp_wz_island,br_brduos,10717821121770145230,1648000,1,wz,156,75,...,NaN,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1650402208,1650403856,mp_wz_island,br_brduos,10717821121770145230,1648000,1,wz,156,75,...,NaN,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1650402208,1650403856,mp_wz_island,br_brduos,10717821121770145230,1648000,1,wz,156,75,...,"{'primaryWeapon': {'name': 's4_ar_asierra44', ...",0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 57 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   utcStartSeconds             156 non-null    int64  
 1   utcEndSeconds               156 non-null    int64  
 2   map                         156 non-null    object 
 3   mode                        156 non-null    object 
 4   matchID                     156 non-null    object 
 5   duration                    156 non-null    int64  
 6   version                     156 non-null    int64  
 7   gameType                    156 non-null    object 
 8   playerCount                 156 non-null    int64  
 9   teamCount                   156 non-null    int64  
 10  draw                        156 non-null    bool   
 11  privateMatch                156 non-null    bool   
 12  kills                       156 non-null    float64
 13  medalXp                     156 non

In [12]:
# make the stats human-readable and parse some values (weapons, games modes)
df_formatted = format_df(df_match,CONF, LABELS)
display(df_formatted)
df_formatted.info()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,version,gameType,playerCount,teamCount,...,loadout_3,missionsComplete,totalMissionXpEarned,totalMissionWeaponXpEarned,assassination,scavenger,domination,supply,timedrun,masterassassination
0,2022-04-19 23:03:28,2022-04-19 23:30:56,mp_wz_island,Duos,10717821121770145230,27,1,wz,156,75,...,NaN,0,0.0,0.0,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
1,2022-04-19 23:03:28,2022-04-19 23:30:56,mp_wz_island,Duos,10717821121770145230,27,1,wz,156,75,...,NaN,0,0.0,0.0,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
2,2022-04-19 23:03:28,2022-04-19 23:30:56,mp_wz_island,Duos,10717821121770145230,27,1,wz,156,75,...,AS44 fists,0,0.0,0.0,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
3,2022-04-19 23:03:28,2022-04-19 23:30:56,mp_wz_island,Duos,10717821121770145230,27,1,wz,156,75,...,NaN,0,0.0,0.0,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
4,2022-04-19 23:03:28,2022-04-19 23:30:56,mp_wz_island,Duos,10717821121770145230,27,1,wz,156,75,...,AS44 fists,1,300.0,300.0,1,<NA>,<NA>,NaN,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2022-04-19 23:03:28,2022-04-19 23:30:56,mp_wz_island,Duos,10717821121770145230,27,1,wz,156,75,...,NaN,2,690.0,390.0,2,<NA>,<NA>,NaN,<NA>,<NA>
152,2022-04-19 23:03:28,2022-04-19 23:30:56,mp_wz_island,Duos,10717821121770145230,27,1,wz,156,75,...,NaN,0,0.0,0.0,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
153,2022-04-19 23:03:28,2022-04-19 23:30:56,mp_wz_island,Duos,10717821121770145230,27,1,wz,156,75,...,NaN,0,0.0,0.0,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
154,2022-04-19 23:03:28,2022-04-19 23:30:56,mp_wz_island,Duos,10717821121770145230,27,1,wz,156,75,...,Bren fists,0,0.0,0.0,<NA>,<NA>,<NA>,NaN,<NA>,<NA>


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 57 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   utcStartSeconds             156 non-null    datetime64[ns]
 1   utcEndSeconds               156 non-null    datetime64[ns]
 2   map                         156 non-null    object        
 3   mode                        156 non-null    object        
 4   matchID                     156 non-null    object        
 5   duration                    156 non-null    object        
 6   version                     156 non-null    int64         
 7   gameType                    156 non-null    object        
 8   playerCount                 156 non-null    int64         
 9   teamCount                   156 non-null    Int64         
 10  draw                        156 non-null    bool          
 11  privateMatch                156 non-null    bool          